<a href="https://colab.research.google.com/github/zhihong1224/RNN_demo/blob/master/RNN_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 读取数据集

In [0]:
import os
import time
import math
import torch
from torch import nn,optim
import zipfile
import numpy as np
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
ROOT='gdrive/My Drive/Colab Notebooks/data'
with zipfile.ZipFile(os.path.join(ROOT,'jaychou_lyrics.txt.zip')) as zin:
  with zin.open('jaychou_lyrics.txt') as f:
    corpus_chars=f.read().decode('utf-8')
corpus_chars[:40]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每'

In [0]:
corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
corpus_chars=corpus_chars[:10000]

In [6]:
corpus_chars[:40]

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每'

# 建立字符索引

In [7]:
idx_to_char=list(set(corpus_chars))
print(len(idx_to_char))
char_to_idx={char:idx for idx,char in enumerate(idx_to_char)}
vocab_size=len(char_to_idx)

1027


In [8]:
# 将训练数据中的每个字符转换为索引
corpus_indices=[char_to_idx[char] for char in corpus_chars]
sample=corpus_indices[:20]
print('chars:',''.join(idx_to_char[idx] for idx in sample))
print('indices:',sample)

chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [630, 787, 994, 555, 847, 410, 42, 630, 787, 371, 446, 249, 426, 759, 317, 682, 42, 630, 787, 371]


# 时序数据采样

In [0]:
def sample_data(corpus_indices,batch_size,seq_len):
  # corpus_indices:列表
  batch_len=len(corpus_indices)//batch_size
  batch_num=(batch_len-1)//seq_len
  all_data=torch.tensor(corpus_indices[:batch_len*batch_size]).view(batch_size,batch_len)
  for i in range(batch_num):
    x=all_data[:,i*seq_len:(i+1)*seq_len]
    y=all_data[:,i*seq_len+1:(i+1)*seq_len+1]
    yield x,y

In [10]:
mini_indices=range(30)
batch_size=2
seq_len=6
dataset=sample_data(mini_indices,batch_size,seq_len)
print(mini_indices)
print()
for X,Y in dataset:
  print(X,Y)

range(0, 30)

tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]]) tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]])
tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]]) tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]])


# RNN from scratch

In [0]:
#one-hot向量
def get_onehot(x,vocab_size):
  # x: torch.tensor (batch_size,seq_len)
  # vocab_size:len of vocab
  bs,sq=x.shape
  result=torch.zeros((bs,sq,vocab_size),device=x.device)
  for i in range(bs):
    for j in range(sq):
      result[i,j,int(x[i,j])]=1
  return result

In [12]:
X=torch.arange(10).view(2,5)
inputs=get_onehot(X,vocab_size)
print(len(inputs),inputs[0].shape)

2 torch.Size([5, 1027])


In [86]:
# 初始化模型参数
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
print('will use',device)

def get_params():
  def _one(shape):
    ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
    return torch.nn.Parameter(ts,requires_grad=True)
  
  W_xh=_one((num_inputs,num_hiddens))
  W_hh=_one((num_hiddens,num_hiddens))
  W_hq=_one((num_hiddens,num_outputs))

  b_h=torch.nn.Parameter(torch.zeros(num_hiddens,device=device),requires_grad=True)
  b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device),requires_grad=True)
  return torch.nn.ParameterList([W_xh,W_hh,b_h,W_hq,b_q])


will use cuda


In [0]:
# 定义模型
def init_rnn_state(batch_size,num_hiddens,device):
  return torch.zeros((batch_size,num_hiddens),device=device)

In [0]:
def rnn(inputs,state,params):
  # inputs:(batch_size,seq_len,vocab_size)
  # state:(batch_size,vocab_size)
  # params:params

  W_xh,W_hh,b_h,W_hq,b_q=params
  H=state
  bs,sq,v=inputs.shape
  result=torch.zeros_like(inputs)
  for i in range(sq):
    X=inputs[:,i,:]  #(batch_size,vocab_size)
    H=torch.tanh(torch.matmul(X,W_xh)+torch.matmul(H,W_hh)+b_h)
    Y=torch.matmul(H,W_hq)+b_q
    result[:,i,:]=Y
  return result,H

In [33]:
state=init_rnn_state(X.shape[0],num_hiddens,device)
inputs=get_onehot(X.to(device),vocab_size)
params=get_params()
outputs,state_new=rnn(inputs,state,params)
print(outputs.shape,state_new.shape)

torch.Size([2, 5, 1027]) torch.Size([2, 256])


In [0]:
# 定义预测函数
def predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,
    device,idx_to_char,char_to_idx):
  output=[char_to_idx[prefix[0]]]
  state=init_rnn_state(1,num_hiddens,device=device) 
  for i in range(num_chars):
    x=get_onehot(torch.tensor([[output[-1]]],device=device),vocab_size)   # (1,1,vocab_size)
    y,state=rnn(x,state,params) # y:(1,1,vocab_size)
    if i<len(prefix)-1:
      output.append(char_to_idx[prefix[i+1]])
    else:
      output.append(int(y[0].argmax(dim=-1).item()))
  return ''.join([idx_to_char[idx] for idx in output])

In [46]:
predict_rnn('分开',10,rnn,params,init_rnn_state,num_hiddens,vocab_size,device,idx_to_char,char_to_idx)

'分开碗它埋钩准喜分温逗'

In [0]:
# 裁剪梯度
def grad_clipping(params,theta,device):
  norm=torch.tensor([0.],device=device)
  for param in params:
    norm+=(param.grad.data**2).sum()
  norm=norm.sqrt().item()
  if norm>theta:
    for param in params:
      param.grad.data*=(theta/norm)

In [0]:
# 训练
def train(rnn,get_params,init_rnn_state,sample_data,corpus_indices,batch_size,
    seq_len,vocab_size,num_hiddens,num_chars,num_epochs,lr,clipping_theta,
    pred_period,idx_to_char,char_to_idx):
  criterion=nn.CrossEntropyLoss()
  params=get_params()
  optimizer=optim.Adam(params,lr=lr)
  
  for epoch in range(num_epochs):
    state=init_rnn_state(batch_size,num_hiddens,device=device)
    train_loss,n=0.0,0
    data_iter=sample_data(corpus_indices,batch_size,seq_len)
    for X,Y in data_iter:
      X=X.cuda()
      Y=Y.cuda()
      inputs=get_onehot(X,vocab_size)  #(batch_size,seq_len,vocab_size)
      outputs,state=rnn(inputs,state,params) #output:(batch_size,seq_len,vocab_size)
      outputs=outputs.view(-1,vocab_size) #(batch_size*seq_len,vocab_size)
      y=Y.view(-1)
      loss=criterion(outputs,y)
      optimizer.zero_grad()
      loss.backward()
      grad_clipping(params,clipping_theta,device)
      optimizer.step()

      if isinstance(state,tuple):
        state=(state[0].detach(),state[1].detach())
      else:
        state=state.detach()

      train_loss+=loss.item()*batch_size*seq_len
      n+=y.shape[0]
    if (epoch+1)%pred_period==0:
      print('epoch:{},train loss:{}'.format(epoch+1,train_loss/n))
      for prefix in prefixes:
        print('-',predict_rnn(prefix,num_chars,rnn,params,init_rnn_state,num_hiddens,vocab_size,
    device,idx_to_char,char_to_idx))



In [0]:
num_epochs,seq_len,batch_size,lr,clipping_theta=250,35,32,0.003,1e-2
num_chars=50
pred_period,prefixes=50,['分开','不分开']

In [76]:
train(rnn,get_params,init_rnn_state,sample_data,corpus_indices,batch_size,
    seq_len,vocab_size,num_hiddens,num_chars,num_epochs,lr,clipping_theta,
    pred_period,idx_to_char,char_to_idx)

epoch:50,train loss:5.2139909863471985
- 分开 我我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的
- 不分开 我我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我
epoch:50,train loss:5.2139909863471985
- 分开 我我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的
- 不分开 我我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我的我  我
epoch:100,train loss:4.3746888637542725
- 分开 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 
- 不分开 我不么的可爱 我的让我你你的可爱 我的让我你你的可爱 我的让我你你的可爱 我的让我你你的可爱 
epoch:100,train loss:4.3746888637542725
- 分开 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 我不么 我 
- 不分开 我不么的可爱 我的让我你你的可爱 我的让我你你的可爱 我的让我你你的可爱 我的让我你你的可爱 
epoch:150,train loss:3.6700417697429657
- 分开 你你着我 我不能的可爱女人 我不了我 我不是我不想的可爱女人 我不了我  在一场的人 我不能我不
- 不分开 我不么的可爱女人 我不了我  在一起的可爱女人 我不了我  在一起 我不能 我不要你的想你你想
epoch:150,train loss:3.6700417697429657
- 分开 你你着我 我不能的可爱女人 我不了我 我不是我不想的可爱女人 我不了我  在一场的人 我不能我不
- 不分开 我不么的可爱女人 我不了我  在一起的可爱女人 我不了我  在一起 我不能 我不要你的想你你想
epoch:200,train loss:3.05916228890419
- 分开 你牵着  想要你 一子我 我不能 我 我不着我 我不能的让我的可不可爱 不要你的想我不想 我不能
- 不分开 我不能我不多 我 一直么  不是

# GRU from scratch

In [0]:
# 参数初始化
def get_params():
  def _one(shape):
    ts=torch.tensor(np.random.normal(0,0.01,shape),device=device,dtype=torch.float32)
    return torch.nn.Parameter(ts,requires_grad=True)
  def _three():
    return (_one((num_inputs,num_hiddens)),
        _one((num_hiddens,num_hiddens)),
        torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),requires_grad=True))
  W_xz,W_hz,b_z=_three()
  W_xr,W_hr,b_r=_three()
  W_xh,W_hh,b_h=_three()

  W_hq=_one((num_hiddens,num_outputs))
  b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
  return torch.nn.ParameterList([W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q])

In [0]:
# 初始化隐藏状态
def init_gru_state(batch_size,num_hiddens,device):
  return torch.zeros((batch_size,num_hiddens),device=device)

In [0]:
# 定义模型
def gru(inputs,state,params):
  # inputs:(batch_size,seq_len,vocab_size)
  # state:(batch_size,num_hiddens)
  W_xz,W_hz,b_z,W_xr,W_hr,b_r,W_xh,W_hh,b_h,W_hq,b_q=params
  H=state
  bs,sq,v=inputs.shape
  results=torch.zeros((bs,sq,v),device=device)
  for i in range(sq):
    X=inputs[:,i,:] #(bs,vocab_size)
    Z=torch.sigmoid(torch.matmul(X,W_xz)+torch.matmul(H,W_hz)+b_z)
    R=torch.sigmoid(torch.matmul(X,W_xr)+torch.matmul(H,W_hr)+b_r)
    H_tilda=torch.tanh(torch.matmul(X,W_xh)+torch.matmul(R*H,W_hh)+b_h)
    H=Z*H+(1-Z)*H_tilda
    Y=torch.matmul(H,W_hq)+b_q   #(batch_size,vocab_size)
    results[:,i,:]=Y   
  return results,H

In [0]:
#训练
num_epochs,seq_len,batch_size,lr,clipping_theta=160,35,32,0.003,1e-2
pred_period,num_chars,prefixes=40,50,['分开','不分开']

In [99]:
train(gru,get_params,init_gru_state,sample_data,corpus_indices,batch_size,
    seq_len,vocab_size,num_hiddens,num_chars,num_epochs,lr,clipping_theta,
    pred_period,idx_to_char,char_to_idx)

epoch:40,train loss:0.5922007896006107
- 分开始打我 别怪我 别怪我 别怪我 别怪我 别怪我 别怪我 别怪我 别怪我 别怪我 别怪我 别怪我 别
- 不分开 不知不觉 我不能再想 我不要再想 我不 我不 我不能再想 我不 我不 我不能再想 我不 我不 
epoch:80,train loss:0.02615923178382218
- 分开 一场悲剧 我的人在抽屉 它所拥有的只剩下回忆 相爱还有别离 像无法被安排的雨 随时准备来袭 我怀
- 不分开  它一直在身边 干什么 干什么 我打开任督二脉 干什么 干什么 东亚病夫的招牌 干什么 干什么
epoch:120,train loss:0.01577853853814304
- 分开 在小村外的溪边河口默默等著我 娘子依旧每日折一枝杨柳 你在那里 在小村外的溪边 默默等待 娘子 
- 不分开 爱情来的太快就像龙卷风 离不开暴风圈来不及逃 我不能再想 我不能再想 我不 我不 我不能 爱情
epoch:160,train loss:0.01548444724176079
- 分开 在小村外的溪边 默默等待 娘子 有什么不妥 有话就直说 别窝在角落 不爽就反驳 到底拽什么 懂不
- 不分开 爱能 到你一口 随风跟著我 一口一口吃掉忧愁 载著你 彷彿载著阳光 不管到哪里都是晴天 蝴蝶自


# LSTM from scratch


In [19]:
num_inputs,num_hiddens,num_outputs=vocab_size,256,vocab_size
device=torch.device('cuda') if torch.cuda.is_available() else 'cpu'
print('will use',device)

def get_params():
  def _one(shape):
    ts=torch.tensor(np.random.normal(0,0.01,size=shape),device=device,dtype=torch.float32)
    return torch.nn.Parameter(ts,requires_grad=True)
  def _three():
    return (_one((num_inputs,num_hiddens)),
        _one((num_hiddens,num_hiddens)),
        torch.nn.Parameter(torch.zeros(num_hiddens,device=device,dtype=torch.float32),
              requires_grad=True))
  W_xi,W_hi,b_i=_three()
  W_xo,W_ho,b_o=_three()
  W_xf,W_hf,b_f=_three()
  W_xc,W_hc,b_c=_three()

  W_hq=_one((num_hiddens,num_outputs))
  b_q=torch.nn.Parameter(torch.zeros(num_outputs,device=device,dtype=torch.float32),requires_grad=True)
  return nn.ParameterList([W_xi,W_hi,b_i,W_xo,W_ho,b_o,W_xf,W_hf,b_f,W_xc,W_hc,b_c,W_hq,b_q])


will use cuda


In [0]:
# 初始化隐藏状态
def init_lstm_state(batch_size,num_hiddens,device):
  return (torch.zeros((batch_size,num_hiddens),device=device),
      torch.zeros((batch_size,num_hiddens),device=device))
      

In [0]:
# 定义模型
def lstm(inputs,state,params):
  # inputs:(batch_size,seq_len,vocab_size)
  # state:(H,C) (batch_size,num_hiddens)
  W_xi,W_hi,b_i,W_xo,W_ho,b_o,W_xf,W_hf,b_f,W_xc,W_hc,b_c,W_hq,b_q=params
  (H,C)=state
  bs,sq,v=inputs.shape
  results=torch.zeros((bs,sq,v),device=inputs.device)
  for i in range(sq):
    X=inputs[:,i,:]   #(batch_size,vocab_size)
    I=torch.sigmoid(torch.matmul(X,W_xi)+torch.matmul(H,W_hi)+b_i)
    O=torch.sigmoid(torch.matmul(X,W_xo)+torch.matmul(H,W_ho)+b_o)
    F=torch.sigmoid(torch.matmul(X,W_xf)+torch.matmul(H,W_hf)+b_f)
    C_tilda=torch.tanh(torch.matmul(X,W_xc)+torch.matmul(H,W_hc)+b_c)
    C=I*C_tilda+F*C
    H=O*torch.tanh(C)
    Y=torch.matmul(H,W_hq)+b_q
    results[:,i,:]=Y
  return results,(H,C)

In [0]:
# 训练模型
num_epochs,seq_len,batch_size,lr,clipping_theta=160,35,32,0.003,1e-2
pred_period,num_chars,prefixes=40,50,['分开','不分开']

In [25]:
train(lstm,get_params,init_lstm_state,sample_data,corpus_indices,batch_size,
    seq_len,vocab_size,num_hiddens,num_chars,num_epochs,lr,clipping_theta,
    pred_period,idx_to_char,char_to_idx)

epoch:40,train loss:3.46826308965683
- 分开始的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
- 不分开始不觉 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不
epoch:80,train loss:1.8293461501598358
- 分开始的见 我想要你的见 我想要你这样打我妈妈 你想你的让我妈妈 你想要你 想要你 想要你 想要你 想
- 不分开始的见 我不能再想 我不要再想 我不要再想 我不能再想 我不能再想 我不能再想 我不能再想 我不
epoch:120,train loss:0.8255559429526329
- 分开始的爸 心伤透明的我面开我 爱过苏美女人 透明的让我感动的可爱女人 坏坏的让我疯狂的可爱女人 坏坏
- 不分开始的微 从小的钟  纪录第一次遇见的你 说你看着我 开始球口 相给还的可以 我给你的可爱女人 坏
epoch:160,train loss:0.3391432352364063
- 分开始的爸  说下午三三四四  回忆的的字迹依然清晰可见 我给你的爱写在西元前 深埋在美索不达米亚平原
- 不分开始的脸 祭司 神殿 征战 弓箭 是谁的从前 喜欢在人切记 盲无能为力再提起 决定中断熟悉 然后在
